### importing require packages

In [1]:
from __future__ import print_function

import json
import os
import numpy as np
import sys

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.engine import Input
from keras.layers import Embedding, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM

from nltk.tokenize import word_tokenize

Using TensorFlow backend.


### tokenizer function

In [2]:
# tokenizer: can change this as needed
# takes input one sentence at a time and returns individual words list
tokenize = lambda x: word_tokenize(x)

### create embeding and store weights for whole vocab

In [3]:
def create_embeddings(data_dir,
                      embeddings_path='lstm2-w2vec/embeddings.npz',
                      vocab_path='lstm2-w2vec/vocab.json',
                      **params):
    """
    Generate embeddings from a batch of text
    :param embeddings_path: where to save the embeddings
    :param vocab_path: where to save the word-index map
    """

    class SentenceGenerator(object):
        """
        Say we want to further preprocess the words from the files — convert to unicode, lowercase, 
        remove numbers, extract named entities… All of this can be done inside the MySentences iterator 
        and word2vec doesn’t need to know. All that is required is that the input yields one 
        sentence (list of utf8 words) after another
        
        """
        
        def __init__(self, dirname):
            self.dirname = os.path.join(os.getcwd(), dirname)
        
        ## iterator to yield tokenized format of one sentence at  a time
        def __iter__(self):
            for fname in os.listdir(self.dirname):
                for line in open(os.path.join(self.dirname, fname)):
                    yield tokenize(line.lower())

    """
    Gensim only requires that the input must provide sentences sequentially, when iterated over. 
    No need to keep everything in RAM: we can provide one sentence, process it, 
    forget it, load another sentence    
    """
    ## provide data directory and it will take sentences one at a time from all the files in directory
        
    sentences = SentenceGenerator(data_dir)
    model = Word2Vec(sentences, **params)
    
    weights = model.wv.syn0
    
    ## storing weights to be later used in keras embedding layer
    np.save(open(embeddings_path, 'wb'), weights)
    
    ## generating word2index
    vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])
    with open(vocab_path, 'w') as f:
        f.write(json.dumps(vocab))
    return model,weights

In [4]:
def load_vocab(vocab_path='lstm2-w2vec/vocab.json'):
    """
    Load word -> index and index -> word mappings
    :param vocab_path: where the word-index map is saved
    :return: word2idx, idx2word
    """

    with open(vocab_path, 'r') as f:
        data = json.loads(f.read())
    word2idx = data
    idx2word = dict([(v, k) for k, v in data.items()])
    return word2idx, idx2word

In [5]:
def word2vec_embedding_layer(embeddings_path='lstm2-w2vec/embeddings.npz'):
    """
    Generate an embedding layer word2vec embeddings
    :param embeddings_path: where the embeddings are saved (as a numpy file)
    :return: the generated embedding layer
    """
    
    weights = np.load(open(embeddings_path, 'rb'))
    layer = Embedding(input_dim=weights.shape[0],
                      output_dim=weights.shape[1],
                      #input_length=100,
                      weights=[weights])
    return layer

In [6]:
# specify embeddings in this environment variable
data_path = 'corpus'

# variable arguments are passed to gensim's word2vec model
w2v_model, weigths_ = create_embeddings(data_path, size=100, min_count=1,
                  window=5, sg=1, iter=25)

In [7]:
word2idx, idx2word = load_vocab()

In [8]:
vocab_size = len(word2idx.keys())
epoch = 1
batch_size = 32

In [9]:
filename = "corpus/alice.txt"
input_txt = open(filename).read()
input_txt = input_txt.lower()

In [10]:
tt = tokenize(input_txt)

In [11]:
x=[]
y=[]
window = 10
n_words = vocab_size
# generating dataset
for i in range(0, n_words - window, 1):
    seq_in = tt[i:i + window]
    seq_out = tt[i + window]
    x.append([word2idx.get(word,) for word in seq_in])
    y.append(weigths_[word2idx[seq_out]])
n_patterns = len(x)
print ("Dataset size : ", n_patterns)

Dataset size :  3052


In [12]:
word2idx

{'feeling': 490,
 'done': 272,
 'ornamented': 1133,
 'thing': 95,
 'execution.’': 1603,
 '‘they’d': 1604,
 'tumbled': 1605,
 'things': 148,
 'three': 181,
 'set': 273,
 'disagree': 1606,
 'sadly': 619,
 'shepherd': 1607,
 'let’s': 1132,
 'pencils': 1608,
 'at': 22,
 'act': 1609,
 'effect': 881,
 'burning': 2554,
 'riddle': 1610,
 'waiting': 407,
 'ours': 1611,
 'knave': 408,
 'bawled': 1612,
 'crashed': 1613,
 'queen’s': 491,
 'fit': 880,
 '“it”': 1135,
 'end.’': 1615,
 'editions': 1137,
 '‘after': 1138,
 'about': 45,
 'falling': 1139,
 'crossly': 1616,
 'splashing': 1140,
 'partner': 1617,
 'both': 314,
 'without': 176,
 'extras': 1618,
 'follow': 1141,
 '‘exactly': 1142,
 'shutting': 1143,
 'lived': 882,
 'seems': 725,
 'history': 492,
 'unhappy': 1144,
 'precious': 1620,
 'returned': 1145,
 'same': 193,
 'night-air': 1622,
 'puppy’s': 1623,
 'apples': 1146,
 'slates’ll': 1624,
 'adventures.’': 1625,
 'frontispiece': 1626,
 'dead': 738,
 'life': 374,
 'ring': 1147,
 'mushroom': 441,


In [13]:
y = np.array(y)

In [14]:
x = np.array(x)

In [15]:
##layer = Embedding(input_dim=weights.shape[0],output_dim=weights.shape[1],weights=[weights])
model = Sequential()
model.add(word2vec_embedding_layer())
model.add(LSTM(1024, return_sequences=True))
#model.add(Dropout(0.2))
model.add(LSTM(512))
#model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         306200    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 1024)        4608000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dense_1 (Dense)              (None, 100)               51300     
Total params: 8,113,276
Trainable params: 8,113,276
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(x,y,batch_size=32,epochs=50,verbose=1)

Epoch 1/50
3052/3052 [==============================] - 75s - loss: 0.0436 - acc: 0.1130    
Epoch 2/50
3052/3052 [==============================] - 80s - loss: 0.0422 - acc: 0.1435    
Epoch 3/50
3052/3052 [==============================] - 77s - loss: 0.0417 - acc: 0.1737    
Epoch 4/50
3052/3052 [==============================] - 79s - loss: 0.0412 - acc: 0.1884    
Epoch 5/50
3052/3052 [==============================] - 76s - loss: 0.0408 - acc: 0.2090    
Epoch 6/50
3052/3052 [==============================] - 75s - loss: 0.0403 - acc: 0.2202    
Epoch 7/50
3052/3052 [==============================] - 75s - loss: 0.0398 - acc: 0.2280    
Epoch 8/50
3052/3052 [==============================] - 77s - loss: 0.0393 - acc: 0.2510    
Epoch 9/50
3052/3052 [==============================] - 75s - loss: 0.0387 - acc: 0.2644    
Epoch 10/50
3052/3052 [==============================] - 75s - loss: 0.0381 - acc: 0.2798    
Epoch 11/50
3052/3052 [==============================] - 75s - loss: 

In [17]:
np.array([x[22]]).shape

(1, 10)

In [18]:
t1 = model.predict(np.array([x[22]]))

In [19]:
t1.shape

(1, 100)

In [20]:
w2v_model.similar_by_vector(t1[0])

[('arrived', 0.266371488571167),
 ('walk', 0.2397647500038147),
 ('far', 0.21389909088611603),
 ('time', 0.2027701437473297),
 ('hour', 0.2004198431968689),
 ('should', 0.19973643124103546),
 ('start', 0.18106308579444885),
 ('6:46', 0.17673756182193756),
 ('3', 0.1729300618171692),
 ('wonderful', 0.17180205881595612)]

In [47]:
start = 380
pattern = list(x[start])
print("\"",' '.join(idx2word[index] for index in pattern))
for i in range(10):
    prediction = model.predict(np.array([pattern]))
    #index = 0 #np.argmax(prediction)
    pred_word = w2v_model.similar_by_vector(prediction[0])[0][0]
    sys.stdout.write(pred_word+" ")
    pattern.append(word2idx[pred_word])
    pattern = pattern[1:len(pattern)]

" to wonder what was going to happen next . first
, she tried to look down and make out what 